In [ ]:
#!/usr/bin/python3
import minizinc
from minizinc import *
from model.Strategy import *
from model.Plate import *
from math import floor, ceil
import numpy as np

class VLSI_CP(Strategy):
    def solve_vlsi(self, plate: Plate) -> Plate:
        (n, (w,), cs) = (plate.get_n(), plate.get_dim(), plate.get_circuits())
        
        csw, csh = [], []
        for i in range(n):
            csw.append(cs[i].get_dim()[0])
            csh.append(cs[i].get_dim()[1])

        # Define upperbound and lowerbound                           
        max_x, max_y = max(csw), max(csh)

        max_block_per_w = floor(w / max_x)      
        A = sum([csw[i] * csh[i] for i in range(n)])
        h_max = ceil(n * max_y / max_block_per_w) + 2
        h_min = max(ceil(A / w), max_y) 

        #CP
        model = Model("./model.mzn")
        solver = Solver.lookup("chuffed")
        
        instance = Instance(solver, model)
        result = instance.solve()
        
        plate.set_dim((result["width"], result["height"])
        for i in range(n):
            plate.get_circuit(i).set_coordinate((result['x['+i+']'], result['y['+i+']']))
        
        return plate